<a href="https://colab.research.google.com/github/rashid200gb/Inventory-Management-System/blob/main/RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.0 MB/s eta 0:00:00


In [14]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
userdata.get('PINECONE-DB')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['Pinecone_DB'] = userdata.get('PINECONE-DB')

In [15]:
# Ensure your VertexAI credentials are configured

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")

[0.04703257977962494,
 -0.04019005596637726,
 -0.02902696467936039,
 -0.026809632778167725,
 0.01892058178782463,
 -8.365760731976479e-05,
 0.03111639805138111,
 -0.019520554691553116,
 0.011491335928440094,
 0.009625785052776337,
 0.04571185261011124,
 0.051709506660699844,
 -0.007854084484279156,
 -0.07627559453248978,
 -0.000736518413759768,
 -0.022592447698116302,
 0.011496763676404953,
 -0.0076109664514660835,
 0.006400744896382093,
 -0.0036826308351010084,
 -8.639021689305082e-05,
 0.007910552434623241,
 -0.031401973217725754,
 -0.027668770402669907,
 0.013148307800292969,
 0.005762958899140358,
 -0.0022430468816310167,
 -0.07029420882463455,
 0.007011014502495527,
 0.07013051956892014,
 -0.047634560614824295,
 0.008311820216476917,
 -0.06021169573068619,
 0.01643129624426365,
 0.042709145694971085,
 -0.047674257308244705,
 0.0342608243227005,
 0.021967323496937752,
 -0.007065100595355034,
 0.00032590707996860147,
 0.013825682923197746,
 -0.08921292424201965,
 -0.0340406857430934

In [18]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

api_key = os.environ.get('Pinecone_DB')
pc = Pinecone(api_key= api_key, environment='us-east-1')
index_name = "quickstart3"

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [19]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = PyPDFLoader("Literature.pdf")
documents = loader.load()


# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'Literature.pdf', 'page': 0}, page_content='Document_30.docx\nby\nSubmission date: \n03-Feb-2021 02:29PM (UTC+0500)\nSubmission ID: \n1500600207\nFile name: \nDocument_30.docx \n(46.9K)\nWord count: \n12730\nCharacter count: \n69845'), Document(metadata={'source': 'Literature.pdf', 'page': 31}, page_content='20\n%\nSIMILARITY INDEX\n20\n%\nINTERNET SOURCES\n0\n%\nPUBLICATIONS\n1\n%\nSTUDENT PAPERS\n1\n20\n%\nExclude quotes\nOff\nExclude bibliography\nOff\nExclude matches\nOff\nDocument_30.docx\nORIGINALITY REPORT\nPRIMARY SOURCES\nrepositori.uji.es\nInternet Source')]


In [20]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content))
    # Define metadata
    metadata = {"text": doc.page_content}
    # Upsert the vector with a unique ID
    index.upsert(vectors=[(doc_id, vector,metadata)])

100%|██████████| 2/2 [00:01<00:00,  1.80it/s]


In [21]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

In [22]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever

)

In [23]:
while True:
  user_input = input("Ask something: ")
  if user_input == "quit":
    break
  else:
    response = qa_chain.run(user_input)
    print(response)

Ask something: research
The provided context adds metadata about a document, "Document_30.docx," submitted on February 3rd, 2021.  The submission includes details such as submission ID (1500600207), file size (46.9KB), word count (12730), and character count (69845).  However, this still doesn't provide information about the research itself, only the submission details.  The original answer remains largely unchanged because this new information is supplementary metadata, not content about the research's topic or findings.

Ask something: framework
The new context provides an originality report suggesting a 20% similarity index with internet sources.  This is still insufficient to determine what *framework* is being discussed.  The report doesn't name any frameworks, only indicates a level of similarity to online sources.  Therefore, the original answer remains the most accurate:  There is no information about a specific framework.

Ask something: define gender inequality in pakistan
Th